In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
Imo=pd.read_csv('/content/Imo.csv')


In [4]:
Imo.head()


,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Latitude,Longitude
0,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-001,ALADINMA UMUELEM HALL,175,905,True,-1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.466191,7.189802
1,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-002,OKWUAKU HALL,243,852,True,-1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.466191,7.189802
2,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-003,UMUNKWO VILLAGE HALL,183,704,True,-1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.466191,7.189802
3,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-004,IBEKU HALL,153,646,True,-1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.468114,7.246407
4,IMO,ABOH MBAISE,ENYIOGUGU,16-01-01-006,EZIALA HALL,209,750,True,-1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.466191,7.189802


In [5]:
from geopy.distance import geodesic

In [6]:
def find_neighbours(row, Imo, radius=1):
    neighbours = []
    for index, other_row in Imo.iterrows():
        distance = geodesic((other_row['Latitude'], other_row['Longitude']),
         (row['Latitude'], row['Longitude'])).km
        if distance <= radius:
            neighbours.append(other_row['PU-Name'])
    return neighbours
Imo['neighbours'] = Imo.apply(find_neighbours, args=(Imo, 1), axis=1)

In [7]:
output_path = 'output_with_neighbours.csv'
Imo.to_csv(output_path, index=False)

In [8]:
from google.colab import files
files.download(output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
parties = ['APC', 'LP', 'PDP', 'NNPP']

In [10]:

def calculate_outlier_score(row):
    outlier_scores = {}
    for party in parties:
        neighbour_votes = Imo[Imo['PU-Name'].isin(row['neighbours'])][party]
        outlier_scores[party] = abs(row[party] - neighbour_votes.mean())
    return outlier_scores
Imo['outlier_scores'] = Imo.apply(calculate_outlier_score, axis=1)

In [11]:
for party in parties:
    Imo[f'outlier_score_{party}'] = Imo['outlier_scores'].apply(lambda x: x[party])

In [12]:

output_path = 'output_with_outlier_scores.csv'
Imo.to_csv(output_path, index=False)


In [13]:
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
top_outliers = {}
for party in parties:
    sorted_df = Imo.sort_values(by=f'outlier_score_{party}', ascending=False)
    top_outliers[party] = sorted_df.head(3)

In [15]:
for party in parties:
    print(f"Top 3 outliers for {party}:")
    for index, row in top_outliers[party].iterrows(): # Removed extra indent here
        print(f"Polling Unit: {row['PU-Name']}, Imo['Outlier Score']: {row[f'outlier_score_{party}']}"),
    print(f"Top 3 outliers for {party}:")
    for index, row in top_outliers[party].iterrows():
        print(f"Polling Unit: {row['PU-Name']}, Imo['Outlier Score']: {row[f'outlier_score_{party}']}")
        print(f"Polling Unit: {row['PU-Name']}, Imo['Outlier Score']: {row[f'outlier_score_{party}']}")
        print("Closest Units:")
        neighbours = row['neighbours']
        neighbour_votes = Imo[Imo['PU-Name'].isin(neighbours)][['PU-Name', party]]
        print(neighbour_votes)
        print("\n")

Top 3 outliers for APC:
Polling Unit: UMUJU VILL. HALL, Imo['Outlier Score']: 498.11538461538464
Polling Unit: UMUOJINMA VILLAGE SQUARE, Imo['Outlier Score']: 401.6
Polling Unit: C.P.S. OHAKPU I, Imo['Outlier Score']: 375.625
Top 3 outliers for APC:
Polling Unit: UMUJU VILL. HALL, Imo['Outlier Score']: 498.11538461538464
Polling Unit: UMUJU VILL. HALL, Imo['Outlier Score']: 498.11538461538464
Closest Units:
                                   PU-Name  APC
588            IKPAOKWARAEBO MARKET SQR. I    0
589                      UMUOBOM TOWN HALL    0
590                       UMUJU VILL. HALL  520
591                        AMAGU VILL. SQ.    2
592                  CENT. SCH. II UMUOBOM    4
594                  COMM. SCH. 4, UMUOBOM    3
595                  COMM. SCH. 4, UMUOBOM    0
596                   ONYENGWU VILLAGE SQ.    1
597                COMM. SCH. III, UMUOBOM    1
598                    COMM. SCH. UMUCHEKE    0
599                      UHUZI VILLAGE SQ.    5
600          

In [16]:
output_path = 'output_with_top_outliers.csv'
Imo.to_csv(output_path, index=False)


In [17]:
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>